In [ ]:
%%capture

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))


In [ ]:
from pyomo.environ import *
!sudo apt-get install -y glpk-utils
import numpy as np
import pandas as pd

## Warehouses
location_data = {
    'Warehouse': [
        'W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10', 'W11', 'W12', 'W13', 'W14', 'W15',
        'W16', 'W17', 'W18', 'W19', 'W20', 'W21', 'W22', 'W23', 'W24', 'W25', 'W26', 'W27', 'W28',
        'W29', 'W30', 'W31', 'W32', 'W33', 'W34', 'W35', 'W36', 'W37', 'W38', 'W39', 'W40', 'W41',
        'W42', 'W43', 'W44', 'W45', 'W46', 'W47', 'W48', 'W49', 'W50', 'W51', 'W52', 'W53'
    ],
    'Latitude': [
        27.59137, 26.85383, 17.20569, 28.56098, 29.25725, 31.91242, 13.02118, 20.60115, 23.03439,
        27.33021, 21.86046, 16.27019, 17.82108, 26.55461, 22.37411, 23.52274, 9.60788, 12.62172,
        27.47615, 12.52546, 20.41729, 24.75631, 24.09555, 25.83937, 23.28316, 23.19029, 24.82048,
        15.69152, 14.54547, 19.55980, 25.35598, 11.67809, 17.70758, 22.53180, 34.06834, 21.32824,
        23.26714, 24.04213, 27.66410, 25.39387, 23.08160, 18.64162, 11.22730, 18.83478, 10.59460,
        21.36683, 26.56603, 14.32428, 26.73609, 20.70395, 25.40305, 31.16756, 23.05970
    ],
    'Longitude': [
        76.43122, 84.19225, 81.53633, 80.49032, 77.91551, 76.05651, 78.39280, 76.26120, 88.81676,
        95.48732, 73.78392, 75.24948, 78.88491, 81.66674, 83.58641, 73.49236, 77.85228, 76.88276,
        79.01114, 79.76196, 86.45314, 78.74415, 87.79077, 86.54676, 80.48695, 71.02871, 93.59155,
        78.36632, 79.55849, 73.63010, 76.39305, 92.71646, 82.50955, 77.09682, 74.79684, 82.51482,
        78.21346, 82.42423, 74.46837, 84.01259, 86.36763, 77.13884, 76.97794, 74.83712, 79.75781,
        80.17494, 93.07447, 76.49978, 91.24404, 78.64692, 73.72323, 77.65145, 76.57799
    ],
    'City': [
        'Roondh Rampur', 'Amwas Khas', 'Rajahmundry', 'Sumer Nagar', 'Bhoomma', 'Kasba',
        'Gennerahalli', 'Taroda', 'Bangaon', 'Lakhipathar', 'Panchmuli', 'Bagalkot', 'Bhongir',
        'Para', 'Raigarh', 'Behdaj', 'Moolippatti', 'K. Gowdagere', 'Khiriya Nagar Shah', 'Kancheepuram',
        'Parakula', 'Gangasagar', 'Karkaria', 'Parari', 'Sakari', 'Kutch', 'Tamenglong', 'Nandyal',
        'Bandarupalli', 'Hinglud', 'Tisaya', 'Port Blair', 'Anakapalle', 'Dewas', 'Batmaloo', 'Dewgaon',
        'Raisen', 'Suhira', 'Nagaur', 'Kasimpur', 'Rupapatia', 'Wanjarwada', 'Coimbatore',
        'Mandavgan', 'Venmanacheri', 'Purgaon', 'Deusur Sang', 'Chitradurga', 'Baksa', 'Wardha',
        'Kaklawas', 'Shimla', 'Kalyanpura'
    ],
    'State': [
        'Rajasthan', 'Uttar Pradesh', 'Andhra Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Himachal Pradesh',
        'Karnataka', 'Maharashtra', 'West Bengal', 'Assam', 'Gujarat', 'Karnataka', 'Telangana', 'Uttar Pradesh',
        'Chhattisgarh', 'Gujarat', 'Tamil Nadu', 'Karnataka', 'Uttar Pradesh', 'Tamil Nadu', 'Odisha', 'Uttar Pradesh',
        'West Bengal', 'Bihar', 'Madhya Pradesh', 'Gujarat', 'Manipur', 'Andhra Pradesh', 'Andhra Pradesh',
        'Maharashtra', 'Rajasthan', 'Andaman and Nicobar', 'Andhra Pradesh', 'Madhya Pradesh', 'Srinagar',
        'Chhattisgarh', 'Madhya Pradesh', 'Madhya Pradesh', 'Rajasthan', 'Bihar', 'West Bengal', 'Maharashtra',
        'Tamil Nadu', 'Maharashtra', 'Tamil Nadu', 'Maharashtra', 'Assam', 'Karnataka', 'Assam', 'Maharashtra',
        'Rajasthan', 'Himachal Pradesh', 'Madhya Pradesh'
    ]
}

# Creating the DataFrame
warehouse_df = pd.DataFrame(location_data)

## Map zones

zone_mapping = {
    'North': {'Srinagar', 'Himachal Pradesh'},
    'North Central': {'Rajasthan', 'Haryana', 'Uttar Pradesh'},
    'Central': {'Madhya Pradesh', 'Chhattisgarh'},
    'West': {'Maharashtra'},
    'South West': {'Karnataka', 'Kerala'},
    'South East': {'Andhra Pradesh', 'Telangana', 'Tamil Nadu', 'Andaman and Nicobar'},
    'East': {'Bihar', 'Jharkhand', 'West Bengal', 'Odisha'},
    'North East': {'Assam', 'Manipur'},
    'North West': {'Gujarat'}
}

# Function to map states to zones
def map_state_to_zone(state):
    for zone, states in zone_mapping.items():
        if state in states:
            return zone
    return 'Unknown'


warehouse_df['Zone'] = warehouse_df['State'].apply(map_state_to_zone)

warehouse_df.head()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


,Warehouse,Latitude,Longitude,City,State,Zone
0,W1,27.59137,76.43122,Roondh Rampur,Rajasthan,North Central
1,W2,26.85383,84.19225,Amwas Khas,Uttar Pradesh,North Central
2,W3,17.20569,81.53633,Rajahmundry,Andhra Pradesh,South East
3,W4,28.56098,80.49032,Sumer Nagar,Uttar Pradesh,North Central
4,W5,29.25725,77.91551,Bhoomma,Uttar Pradesh,North Central


In [ ]:
## Plants
plant_data = {
    'Plant': [
        'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8'
    ],
    'Latitude': [
        26.1158, 30.3165, 30.9578, 21.0077, 10.7976, 11.2746, 11.9416, 22.9920
    ],
    'Longitude': [
        91.7086, 78.0322, 76.7914, 75.5626, 76.7430, 77.5827, 79.8083, 72.3773
    ],
    'City': [
        'Guwuhati', 'Dehradun', 'Baddi', 'Jalgaon', 'Kanjikode', 'Perundara', 'Pondicherry', 'Sanand'
    ],
    'State': [
        'Assam', 'Uttarakahand', 'Himachal Pradesh', 'Maharashtra', 'Kerala', 'Tamil Nadu', 'Tamil Nadu', 'Gujarat'
    ],
    'Intercept': [
        2.73, 3.84, 3.84, 2.52, 1.64, 1.76, 2.1, 2.85
        ],

    'Coefficient': [
        0.002, 0.0052, 0.0052, 0.0048, 0.0042, 0.0043, 0.0044, 0.005
        ],

}

plant_product = {
    'P1' : {'CNO': 0, 'Edible Oil': 0, 'VAHO': 1},
    'P2' : {'CNO': 0, 'Edible Oil': 0, 'VAHO': 1},
    'P3': {'CNO': 0, 'Edible Oil': 1, 'VAHO': 0},
    'P4' : {'CNO': 0, 'Edible Oil': 1, 'VAHO': 0},
    'P5' : {'CNO': 1, 'Edible Oil': 0, 'VAHO': 0},
    'P6' : {'CNO': 1, 'Edible Oil': 0, 'VAHO': 0},
    'P7' : {'CNO': 1, 'Edible Oil': 0, 'VAHO': 0},
    'P8' : {'CNO': 0, 'Edible Oil': 0, 'VAHO': 1}
}


plant_df = pd.DataFrame(plant_data)
plant_df

,Plant,Latitude,Longitude,City,State,Intercept,Coefficient
0,P1,26.1158,91.7086,Guwuhati,Assam,2.73,0.0020
1,P2,30.3165,78.0322,Dehradun,Uttarakahand,3.84,0.0052
2,P3,30.9578,76.7914,Baddi,Himachal Pradesh,3.84,0.0052
3,P4,21.0077,75.5626,Jalgaon,Maharashtra,2.52,0.0048
4,P5,10.7976,76.7430,Kanjikode,Kerala,1.64,0.0042
5,P6,11.2746,77.5827,Perundara,Tamil Nadu,1.76,0.0043
6,P7,11.9416,79.8083,Pondicherry,Tamil Nadu,2.10,0.0044
7,P8,22.9920,72.3773,Sanand,Gujarat,2.85,0.0050


In [ ]:
## Calculate zone centroids using the mean of co-ordinates

zone_centroids = warehouse_df.groupby('Zone').agg({
    'Latitude': 'mean',
    'Longitude': 'mean'
}).reset_index()

demand_distribution = {
    'South West': 75690, 'West': 168200, 'South East': 134560, 'North Central': 134560,
    'North West': 92510, 'East': 117740, 'North East': 25230, 'Central': 84100, 'North': 8410
}

product_demand_distribution = {
    'CNO': 0.46, 'VAHO': 0.24, 'Edible Oil': 0.30
}

secondary_freight = {
    'North East': 0.0145,
    'North': 0.011875,
    'West': 0.0115,
    'South West': 0.010625,
    'South East': 0.009125,
    'Central': 0.008875,
    'East': 0.008,
    'North Central': 0.007875,
    'North West': 0.006875
}

In [ ]:
## Zones and demands

zones = {
    'Distributor': ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'],
    'Zone': ['Central', 'East', 'North', 'North Central', 'North East', 'North West',  'South East', 'South West', 'West'],
    'Latitude': zone_centroids['Latitude'].tolist(),
    'Longitude': zone_centroids['Longitude'].tolist(),
    'Net Demand': [84100, 117740, 8410, 134560, 25230, 92510, 134560, 75690, 168200]
}

zone_df = pd.DataFrame(zones)

zone_df['Secondary Freight'] = zone_df['Zone'].map(secondary_freight)

for product, fraction in product_demand_distribution.items():
    zone_df[product] = zone_df['Net Demand'] * fraction

zone_df.head()


,Distributor,Zone,Latitude,Longitude,Net Demand,Secondary Freight,CNO,VAHO,Edible Oil
0,D1,Central,22.840897,80.128669,84100,0.008875,38686.0,20184.0,25230.0
1,D2,East,23.643678,86.664608,117740,0.008000,54160.4,28257.6,35322.0
2,D3,North,32.382773,76.168267,8410,0.011875,3868.6,2018.4,2523.0
3,D4,North Central,26.947363,78.303598,134560,0.007875,61897.6,32294.4,40368.0
4,D5,North East,26.363202,93.349345,25230,0.014500,11605.8,6055.2,7569.0


In [ ]:
## Function to calculate distances provided co-ordinates
def distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius of the Earth in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c
    return distance

In [ ]:
## Calculate distances between plant and warehouse, warehouse and distributor
import numpy as np
P2W_distances = {}

W2D_distances = {}


# Calculate distances between all combinations of locations
for i, row1 in plant_df.iterrows():
    for j, row2 in warehouse_df.iterrows():
        dist = distance(row1['Latitude'], row1['Longitude'], row2['Latitude'], row2['Longitude'])
        P2W_distances[(row1['Plant'], row2['Warehouse'])] = dist

for j, warehouse in warehouse_df.iterrows():
  for k, zone in zone_df.iterrows():
     dist = distance(warehouse['Latitude'], warehouse['Longitude'], zone['Latitude'], zone['Longitude'])
     W2D_distances[(warehouse['Warehouse'], zone['Distributor'])] = dist

data1 = [{'Plant': plant, 'Warehouse': warehouse, 'Distance': distance}
        for (plant, warehouse), distance in P2W_distances.items()]

data2 = [{'Warehouse': warehouse, 'Distributor': distributor, 'Distance': distance}
        for (warehouse, distributor), distance in W2D_distances.items()]


P2W_df = pd.DataFrame(data1)
W2D_df = pd.DataFrame(data2)

In [ ]:
## Calculate Primary costs
P2W_costs = pd.merge(plant_df, P2W_df, left_on='Plant', right_on='Plant')
P2W_costs['Cost'] = P2W_costs['Intercept'] + P2W_costs['Coefficient'] * P2W_costs['Distance']
P2W_costs.head()

,Plant,Latitude,Longitude,City,State,Intercept,Coefficient,Warehouse,Distance,Cost
0,P1,26.1158,91.7086,Guwuhati,Assam,2.73,0.002,W1,1523.444393,5.776889
1,P1,26.1158,91.7086,Guwuhati,Assam,2.73,0.002,W2,752.439813,4.234880
2,P1,26.1158,91.7086,Guwuhati,Assam,2.73,0.002,W3,1443.263932,5.616528
3,P1,26.1158,91.7086,Guwuhati,Assam,2.73,0.002,W4,1140.450088,5.010900
4,P1,26.1158,91.7086,Guwuhati,Assam,2.73,0.002,W5,1401.328060,5.532656


In [ ]:
## Calculate secondary costs

W2D_costs = pd.merge(zone_df, W2D_df, left_on='Distributor', right_on='Distributor')
W2D_costs['Cost'] = W2D_costs['Secondary Freight'] * W2D_costs['Distance']
W2D_costs.head()

,Distributor,Zone,Latitude,Longitude,Net Demand,Secondary Freight,CNO,VAHO,Edible Oil,Warehouse,Distance,Cost
0,D1,Central,22.840897,80.128669,84100,0.008875,38686.0,20184.0,25230.0,W1,645.939709,5.732715
1,D1,Central,22.840897,80.128669,84100,0.008875,38686.0,20184.0,25230.0,W2,605.890268,5.377276
2,D1,Central,22.840897,80.128669,84100,0.008875,38686.0,20184.0,25230.0,W3,643.613838,5.712073
3,D1,Central,22.840897,80.128669,84100,0.008875,38686.0,20184.0,25230.0,W4,637.074073,5.654032
4,D1,Central,22.840897,80.128669,84100,0.008875,38686.0,20184.0,25230.0,W5,746.878170,6.628544


## Mathematical Formulation of the Supply Chain Optimization Problem

### Sets and Indices
- $i \in I$: index and set of factories
- $j \in J$: index and set of warehouses
- $k \in K$: index and set of distributors
- $p \in P$: index and set of products

### Parameters
- $d_{ij}$: distance from factory $i$ to warehouse $j$
- $d_{jk}$: distance from warehouse $j$ to distributor $k$
- $\text{demand}_{kp}$: demand of product $p$ at distributor $k$
- $M$: a large constant used for modeling logical constraints

### Decision Variables
- $x_{ij}^p$: continuous variable representing the amount of product $p$ shipped from factory $i$ to warehouse $j$
- $y_{jk}^p$: continuous variable representing the amount of product $p$ shipped from warehouse $j$ to distributor $k$
- $z_j$: binary variable where $z_j = 1$ if warehouse $j$ is open; otherwise $0$

### Objective Function
Minimize the total transportation cost from factories to warehouses and then from warehouses to distributors:
$$
\text{Minimize } Z = \sum_{i \in I} \sum_{j \in J} \sum_{p \in P} \left( \text{(Primary Freight Intercept)}_i + \text{Primary Freight Coefficient}_i \times d_{ij} \right) \times x_{ij}^p + \sum_{j \in J} \sum_{k \in K} \sum_{p \in P} \text{(Secondary Freight Coefficient)}_j \times d_{jk} \times y_{jk}^p
$$

### Constraints

**Demand Satisfaction at Distributors:**
$$
\sum_{j \in J} y_{jk}^p = \text{demand}_{kp} \quad \forall k \in K, p \in P
$$


**Flow Conservation at Warehouses**
The total amount of each commodity received at each warehouse must equal the total amount shipped to demand zones.

$$
\sum_{i \in I} x_{ij}^p = \sum_{k \in K} y_{jk}^p \quad \forall j, p
$$


**Fixed Number of Warehouses:**
$$
\sum_{j \in J} z_j = 22 \space (or \space 24)
$$

**Capacity Constraints:**

M is a large constant
$$
x_{ij}^p \leq M z_{j} \quad \forall i \in I, j \in J, p \in P
$$
$$
y_{jk}^p \leq M z_{j} \quad \forall j \in J, k \in K, p \in P
$$

**Constraint to ensure 2-day service level**
$$
\frac{\sum_{j \in J} \sum_{k \in K} \sum_{p \in P} d_{jk} \times y_{jkp}}{\text{TotalDemand}} \leq 500
$$

**Positive x and y:**

$$
x_{ij}^p \ge 0,
$$

$$
y_{ij}^p \ge 0
$$

In [ ]:
## Model initialization

model = ConcreteModel()

# Sets
model.I = Set(initialize = [p for p in plant_df['Plant']])  # Plants
model.J = Set(initialize = [w for w in warehouse_df['Warehouse']])  # Warehouses
model.K = Set(initialize = [d for d in zone_df['Distributor']])  # Distributors
model.P = Set(initialize = ['CNO', 'Edible Oil', 'VAHO'])  # Products

# Parameters
model.d_ij = Param(model.I, model.J, initialize = {(i, j): P2W_df.loc[(P2W_df['Plant'] == i) & (P2W_df['Warehouse'] == j), 'Distance'].values[0]
                                                 for i in model.I for j in model.J})  # Distance from Plants to Warehouses
model.d_jk = Param(model.J, model.K, initialize = {(j, k): W2D_df.loc[(W2D_df['Warehouse'] == j) & (W2D_df['Distributor'] == k), 'Distance'].values[0]
                                                 for j in model.J for k in model.K})  # Distance from Warehouses to Distributors
#model.demand_kp = Param(model.K, model.P, initialize = {(k, p): demand[k][p] for k in model.K for p in model.P})
model.demand_kp = Param(model.K, model.P, initialize = {(k, p): zone_df.loc[zone_df['Distributor'] == k][p].values[0] for k in model.K for p in model.P}, mutable = True) # Demand at Distributors

model.FactoryCanProduce = Param(model.I, model.P, initialize = {(i, p): plant_product[i][p] for i in model.I for p in model.P}, within = Binary)

# Primary freight costs (Plant to Warehouse)
model.primary_intercept = Param(model.I, model.J, initialize = {(i, j): P2W_costs.loc[(P2W_costs['Plant'] == i) & (P2W_costs['Warehouse'] == j), 'Intercept'].values[0]
                                                              for i in model.I for j in model.J}, mutable = True)
model.primary_coefficient = Param(model.I, model.J, initialize = {(i, j): P2W_costs.loc[(P2W_costs['Plant'] == i) & (P2W_costs['Warehouse'] == j), 'Coefficient'].values[0]
                                                                for i in model.I for j in model.J}, mutable = True)

# Secondary freight costs (Warehouse to Distributor)
model.secondary_freight = Param(model.J, model.K, initialize = {(j, k): W2D_costs.loc[(W2D_costs['Warehouse'] == j) & (W2D_costs['Distributor'] == k), 'Secondary Freight'].values[0]
                                                              for j in model.J for k in model.K}, mutable = True)

model.numWarehouses = Param(initialize = 22)


## Total demand for each product
total_demand = sum(zone_df.loc[zone_df['Distributor'] == k][p].values[0] for k in model.K for p in model.P)

model.TotalDemand = Param(initialize = total_demand)


# Decision Variables
model.x = Var(model.I, model.J, model.P, within=NonNegativeReals)  # Quantity shipped from plant i to warehouse j of product p
model.y = Var(model.J, model.K, model.P, within=NonNegativeReals)  # Quantity shipped from warehouse j to distributor k of product p
model.z = Var(model.J, within = Binary)  # Binary variable for whether warehouse j is open

In [ ]:
## Objective
def objective_rule(model):
    # Calculate primary shipping costs from plants to warehouses using intercept and coefficient
    primary_costs = sum((model.primary_intercept[i, j] + model.primary_coefficient[i, j] * model.d_ij[i, j]) * model.x[i, j, p]
                        for i in model.I for j in model.J for p in model.P)

    # Calculate secondary shipping costs from warehouses to distributors using the secondary freight rate
    secondary_costs = sum(model.secondary_freight[j, k] * model.d_jk[j, k] * model.y[j, k, p]
                          for j in model.J for k in model.K for p in model.P)

    return primary_costs + secondary_costs

# Set the objective
model.cost = Objective(rule = objective_rule, sense = minimize)

In [ ]:
## Constraints

## Demand must be fulfiled for each zone

def demand_fulfillment(model, k, p):
    return sum(model.y[j, k, p] for j in model.J) == model.demand_kp[k, p]

model.demandFulfillment = Constraint(model.K, model.P, rule = demand_fulfillment)

## Product outflow from warehouse = product inflow at warehouse
def flow_conservation(model, j, p):
    return sum(model.y[j, k, p] for k in model.K) == sum(model.x[i, j, p] for i in model.I)

model.flowConservation = Constraint(model.J, model.P, rule = flow_conservation)

## Constrain the number of operable warehouses = 22

def warehouse_operation(model):
    return sum(model.z[j] for j in model.J) == model.numWarehouses

model.warehouseOperation = Constraint(rule = warehouse_operation)

# Large constant for enforcing conditional constraints
M = 10000000000000

# Constraints for enforcing warehouse active/not dependency with a large constant M

def capacity_dependency_W2D(model, j, k, p):
    return model.y[j, k, p] <= M * model.z[j]

model.capacityDependencyW2D = Constraint(model.J, model.K, model.P, rule = capacity_dependency_W2D)

def capacity_dependency_P2W(model, i, j, p):
    return model.x[i, j, p] <= M * model.z[j]

model.capacityDependencyF2W = Constraint(model.I, model.J, model.P, rule = capacity_dependency_P2W)

## Production constraints as certain plants produce certain products

def plant_product_capability(model, i, j, p):
    return model.x[i, j, p] <= 1000000000 * model.FactoryCanProduce[i, p]  # Large upper bound to essentially make it non-binding if factory can produce the product

model.factoryProductCapability = Constraint(model.I, model.J, model.P, rule = plant_product_capability)

## Average distance <= 500 km for 2 day service

def average_distance(model):
    return sum(model.d_jk[j, k] * model.y[j, k, p] for j in model.J for k in model.K for p in model.P) <= 500 * model.TotalDemand

model.averageDistance = Constraint(rule = average_distance)

In [ ]:
solver = SolverFactory('glpk')

result = solver.solve(model)

# Check the status of the solution
print("Solver Status:", result.solver.status)
print("Solver Termination Condition:", result.solver.termination_condition)

Solver Status: ok
Solver Termination Condition: optimal


In [ ]:
# Objective value
if result.solver.status == 'ok' and result.solver.termination_condition == 'optimal':
    print("Objective Value: Minum achievable freight cost for 22 Warehouses = ₹{:.2f}".format(model.cost()))

    # Warehouses that are open
    print("Warehouses that are open:")
    for j in model.J:
        if model.z[j].value > 0.5:
            print("Warehouse {} is open".format(j))
else:
    print("No optimal solution found. Status:", result.solver.status)

Objective Value: Minum achievable freight cost for 22 Warehouses = ₹5702969.10
Warehouses that are open:
Warehouse W6 is open
Warehouse W8 is open
Warehouse W9 is open
Warehouse W10 is open
Warehouse W11 is open
Warehouse W12 is open
Warehouse W13 is open
Warehouse W14 is open
Warehouse W15 is open
Warehouse W16 is open
Warehouse W17 is open
Warehouse W18 is open
Warehouse W19 is open
Warehouse W20 is open
Warehouse W22 is open
Warehouse W25 is open
Warehouse W29 is open
Warehouse W41 is open
Warehouse W42 is open
Warehouse W46 is open
Warehouse W47 is open
Warehouse W48 is open


In [ ]:
## Check for One-day service level i.e distance = 250km
selected_distances = []
threshold_distance = 250  # Threshold distance in kilometers

selected_warehouses = [j for j in model.J if value(model.z[j]) > 0.5]

for warehouse in selected_warehouses:
    for distributor in model.K:
        distance = value(model.d_jk[warehouse, distributor])
        if distance <= threshold_distance:
            selected_distances.append((warehouse, distributor, distance))

total_selected_warehouses = len(selected_warehouses)
selected_within_250km = len(set([s[0] for s in selected_distances]))  # Count unique warehouses


within_250km = selected_within_250km *100 / total_selected_warehouses
print(f"The percentage of selected warehouses within 250 km: {within_250km:.2f}%")

The percentage of selected warehouses within 250 km: 77.27%


In [ ]:
## Zone-wise one day service level
Wr = []
for i in model.J:
  if model.z[i].value>0:
    Wr.append(i)

print(
    "1 Day service level by zone (in %):"
)
W2D_costs[(W2D_costs['Warehouse'].isin(Wr)) & (W2D_costs['Distance']<=250)].groupby('Zone')['Zone'].count() * 100 / warehouse_df[(warehouse_df['Warehouse'].isin(Wr))].groupby('Zone')['Zone'].count()


1 Day service level by zone (in %):


Zone
Central          100.000000
East             100.000000
North            100.000000
North Central     66.666667
North East       100.000000
North West       100.000000
South East        50.000000
South West        66.666667
West              66.666667
Name: Zone, dtype: float64

In [ ]:
warehouse_df[(warehouse_df['Warehouse'].isin(Wr))]

,Warehouse,Latitude,Longitude,City,State,Zone
5,W6,31.91242,76.05651,Kasba,Himachal Pradesh,North
7,W8,20.60115,76.26120,Taroda,Maharashtra,West
8,W9,23.03439,88.81676,Bangaon,West Bengal,East
9,W10,27.33021,95.48732,Lakhipathar,Assam,North East
10,W11,21.86046,73.78392,Panchmuli,Gujarat,North West
11,W12,16.27019,75.24948,Bagalkot,Karnataka,South West
12,W13,17.82108,78.88491,Bhongir,Telangana,South East
13,W14,26.55461,81.66674,Para,Uttar Pradesh,North Central
14,W15,22.37411,83.58641,Raigarh,Chhattisgarh,Central
15,W16,23.52274,73.49236,Behdaj,Gujarat,North West


In [ ]:
## Sensititvity analysis of cost vs warehouse to determine crucial warehouses

active_warehouses = [j for j in model.J if value(model.z[j]) > 0.5]
print("Active Warehouses:", active_warehouses)

original_cost = value(model.cost)  # Record the original objective value

key_warehouse_impacts = {}

for w in active_warehouses:
    model.z[w].fix(0)  # Temporarily close one warehouse
    solver.solve(model)
    new_cost = value(model.cost)
    model.z[w].unfix()  # Reopen the warehouse for the next iteration

    cost_increase = new_cost - original_cost
    key_warehouse_impacts[w] = cost_increase

    print(f"Closing warehouse {w} increases costs by ₹{cost_increase:.2f}")

Active Warehouses: ['W6', 'W8', 'W9', 'W10', 'W11', 'W12', 'W13', 'W14', 'W15', 'W16', 'W17', 'W18', 'W19', 'W20', 'W22', 'W25', 'W29', 'W41', 'W42', 'W46', 'W47', 'W48']
Closing warehouse W6 increases costs by ₹10588.13
Closing warehouse W8 increases costs by ₹153391.55
Closing warehouse W9 increases costs by ₹-0.00
Closing warehouse W10 increases costs by ₹-0.00
Closing warehouse W11 increases costs by ₹29494.75
Closing warehouse W12 increases costs by ₹-0.00
Closing warehouse W13 increases costs by ₹0.00
Closing warehouse W14 increases costs by ₹-0.00
Closing warehouse W15 increases costs by ₹0.00
Closing warehouse W16 increases costs by ₹13052.01
Closing warehouse W17 increases costs by ₹-0.00
Closing warehouse W18 increases costs by ₹-0.00
Closing warehouse W19 increases costs by ₹19045.09
Closing warehouse W20 increases costs by ₹43152.88
Closing warehouse W22 increases costs by ₹2692.38
Closing warehouse W25 increases costs by ₹30550.39
Closing warehouse W29 increases costs by ₹

In [ ]:
# Solution - Plant to Warehouse shipments

P2W_data = {
    'Plant': [],
    'Warehouse': [],
    'Product': [],
    'Shipment Quantity': [],
    'Distance': []
}

for i in model.I:
    for j in model.J:
        for p in model.P:
            if model.x[i, j, p].value > 1e-6:
                P2W_data['Plant'].append(i)
                P2W_data['Warehouse'].append(j)
                P2W_data['Product'].append(p)
                P2W_data['Shipment Quantity'].append(model.x[i, j, p].value),
                P2W_data['Distance'].append(model.d_ij[i, j])

P2W = pd.DataFrame(P2W_data)
print("\nPlant to Warehouse Shipments:")
P2W


Plant to Warehouse Shipments:


,Plant,Warehouse,Product,Shipment Quantity,Distance
0,P1,W18,VAHO,18165.6,2157.167083
1,P1,W19,VAHO,32294.4,1268.751381
2,P1,W25,VAHO,20184.0,1176.088729
3,P1,W29,VAHO,32294.4,1803.258919
4,P1,W41,VAHO,28257.6,636.610373
5,P1,W47,VAHO,6055.2,145.022300
6,P2,W6,VAHO,2018.4,258.573924
7,P3,W6,Edible Oil,2523.0,126.998369
8,P3,W19,Edible Oil,40368.0,443.008440
9,P4,W8,Edible Oil,50460.0,85.537311


In [ ]:
# Solution - Warehouse to Distributor shipments

W2D_data = {
    'Warehouse': [],
    'Distributor': [],
    'Product': [],
    'Shipment Quantity': [],
    'Distance': []
}

for j in model.J:
    for k in model.K:
        for p in model.P:
            if model.y[j, k, p].value > 1e-6:  # Assuming a small threshold to ignore negligible shipments
                W2D_data['Warehouse'].append(j)
                W2D_data['Distributor'].append(k)
                W2D_data['Product'].append(p)
                W2D_data['Shipment Quantity'].append(model.y[j, k, p].value),
                W2D_data['Distance'].append(model.d_jk[j, k])

W2D = pd.DataFrame(W2D_data)
print("\nWarehouse to Distributor Shipments:")
W2D


Warehouse to Distributor Shipments:


,Warehouse,Distributor,Product,Shipment Quantity,Distance
0,W6,D3,CNO,3868.6,53.348714
1,W6,D3,Edible Oil,2523.0,53.348714
2,W6,D3,VAHO,2018.4,53.348714
3,W8,D9,Edible Oil,50460.0,90.365911
4,W8,D9,VAHO,40368.0,90.365911
5,W11,D6,CNO,42554.6,152.335961
6,W11,D6,Edible Oil,27753.0,152.335961
7,W12,D8,Edible Oil,22707.0,294.244171
8,W16,D6,VAHO,22202.4,104.607584
9,W18,D8,CNO,34817.4,160.441641


In [ ]:
## Quantity of product needed to be produced by each plant

## Note: Assumption, the plants do not have a production capacity constraint; we have supply = demand and supply is distributed across plants without considering production capacity

P2W.groupby(['Plant', 'Product'])['Shipment Quantity'].sum().reset_index()

,Plant,Product,Shipment Quantity
0,P1,VAHO,137251.2
1,P2,VAHO,2018.4
2,P3,Edible Oil,42891.0
3,P4,Edible Oil,209409.0
4,P5,CNO,108320.8
5,P6,CNO,112189.4
6,P7,CNO,166349.8
7,P8,VAHO,62570.4


Demand elasticity analysis for 22 Warehouses

In [ ]:
demand_growth_factors = np.arange(1.05, 1.25, 0.05)

results = []

# Solver setup
solver = SolverFactory('glpk')

for factor in demand_growth_factors:
    # Update demand parameter
    for (k, p) in model.demand_kp:
        original_demand = model.demand_kp[k, p]  # Store original demand
        model.demand_kp[k, p] = original_demand * factor  # Scale demand by the current factor

    # Solve the model with updated demand
    solver.solve(model)

    # Calculate the total distribution cost
    total_cost = value(model.cost)

    # Record the results
    results.append((factor, total_cost))

    # Reset the demand to original for the next iteration
    for (k, p) in model.demand_kp:
        model.demand_kp[k, p] = original_demand

# Print the results
for factor, cost in results:
    print(f"Demand increase by {factor - 1:.0%}: Total Cost = ₹{cost:.2f}")


Demand increase by 5%: Total Cost = ₹5988117.55
Demand increase by 10%: Total Cost = ₹8703239.51
Demand increase by 15%: Total Cost = ₹10008725.43
Demand increase by 20%: Total Cost = ₹12010470.52
